In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
from pytest import approx
import pickle

In [2]:
MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [159]:
new_train = train 
new_train2 = pd.merge(new_train,MV_users.loc[:,['uID','gender','age']], on='uID', how='left')
new_train2['gender'] = [ 0 if x=='F' else 1 for x in new_train2['gender']]
new_train2 = pd.merge(new_train2,MV_movies.loc[:,['mID']+MV_movies.columns.to_list()[2:] ], on='mID', how='left')
new_train2.head()

,uID,mID,rating,gender,age,year,Doc,Com,Hor,Adv,...,Chi,Cri,Thr,Sci,Mys,Rom,Fil,Fan,Act,Mus
0,744,1210,5,1,25,1983,0,0,0,1,...,0,0,0,1,0,1,0,0,1,0
1,3040,1584,4,1,25,1997,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1451,1293,5,1,35,1982,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5455,3176,2,0,18,1999,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4,2507,3074,5,1,25,1972,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
def rmse(matrix1, matrix2):
    return np.sqrt(np.mean((matrix1 - matrix2) ** 2))

In [161]:
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [204]:
model = NMF(n_components=5, init='random', random_state=0)
W = model.fit_transform(new_train2.iloc[:,3:])
H = model.components_
result = np.dot(W,H)

In [205]:
df= pd.DataFrame({'pred':W.argmax(1)})
pred = W.argmax(1)
df.groupby('pred').size()
df2 = pd.DataFrame({'rating':new_train2['rating'],'pred':pred})
df2.groupby(['rating','pred'],as_index=False).size().sort_values(by='size',ascending=False)

,rating,pred,size
19,4,4,231732
14,3,4,173234
24,5,4,149651
9,2,4,70634
4,1,4,36435
15,4,0,11876
10,3,0,9077
20,5,0,8398
5,2,0,4314
0,1,0,2844


This code is to get the best labels 

In [206]:
import itertools
elements= [0,1, 2, 3,4]
perms = itertools.permutations([1,2,3,4,5],5)
results = []
for x in perms:
    mapping = dict( [ z for z in zip(elements, x)])
    df2['pred2'] = [mapping[y] for y in df2['pred']]
    results.append((x,rmse(df2['rating'],df2['pred2'])))

In [207]:
m = min([x[1] for x in results])
best = None
for x in results:
    if x[1]==m:
        best= x[0]
best = dict( [z for z in zip(elements,best)])

For trainign we have a RMSE of ~ 1.97

In [209]:
df2['pred2'] = [best[y] for y in df2['pred']]

rmse(df2['rating'],df2['pred2'])

1.1969239362819357

## Using Test data

We first create the test data matrix 

In [212]:
new_test = test 
new_test2 = pd.merge(new_test,MV_users.loc[:,['uID','gender','age']], on='uID', how='left')
new_test2['gender'] = [ 0 if x=='F' else 1 for x in new_test2['gender']]
new_test2 = pd.merge(new_test2,MV_movies.loc[:,['mID']+MV_movies.columns.to_list()[2:] ], on='mID', how='left')
new_test2.head()

,uID,mID,rating,gender,age,year,Doc,Com,Hor,Adv,...,Chi,Cri,Thr,Sci,Mys,Rom,Fil,Fan,Act,Mus
0,2233,440,4,0,35,1993,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,4274,587,5,0,45,1990,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
2,2498,454,3,1,56,1993,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,2868,2336,5,1,35,1998,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1636,2686,5,0,25,1998,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Now, with our trained model, we get the transformation:

In [213]:
W = model.transform(new_test2.iloc[:,3:])
df= pd.DataFrame({'pred':W.argmax(1)})
pred = W.argmax(1)
df.groupby('pred').size()
df2 = pd.DataFrame({'rating':new_test2['rating'],'pred':pred})
df2.groupby(['rating','pred'],as_index=False).size().sort_values(by='size',ascending=False)

,rating,pred,size
7,4,4,83961
5,3,4,62342
9,5,4,53482
3,2,4,24791
6,4,0,20785
4,3,0,16053
8,5,0,14319
1,1,4,12174
2,2,0,7592
0,1,0,4564


For the test dataset we get a RMSE of 1.19

In [215]:
df2['pred2'] = [best[y] for y in df2['pred']]

rmse(df2['rating'],df2['pred2'])

1.194951939983712